In [ ]:
# Importación de Librerías
import pandas as pd
import psycopg2 
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("002_eda_airbnb.log"),
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook de EDA (002_eda_airbnb.ipynb).")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100) 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 1000)

logging.info("Configuraciones de Pandas para visualización aplicadas.")

# Extracción de Datos desde los archivos CSV
# Definición de Variables Constantes y Carga de .env
CSV_FILE_PATH = '../../data/raw/Airbnb_Open_Data.csv'

logging.info(f"Ruta del archivo CSV: {CSV_FILE_PATH}")

# Pre-definición del DataFrame
df_airbnb = pd.DataFrame()
logging.info("DataFrame 'df_airbnb' predefinido como un DataFrame vacío.")

# Carga del DataFrame desde el archivo CSV
try:
    logging.info(f"Intentando cargar el archivo CSV: {CSV_FILE_PATH}")
    df_airbnb = pd.read_csv(CSV_FILE_PATH, low_memory=False)
    logging.info(f"Archivo CSV '{CSV_FILE_PATH}' cargado exitosamente.")
    logging.info(f"El DataFrame tiene {df_airbnb.shape[0]} filas y {df_airbnb.shape[1]} columnas.")
except FileNotFoundError:
    logging.error(f"Error: Archivo CSV no encontrado en '{CSV_FILE_PATH}'")
    raise
except Exception as e:
    logging.error(f"Ocurrió un error al cargar el CSV '{CSV_FILE_PATH}': {e}")
    raise

# Verificación de la carga del DataFrame
if not df_airbnb.empty:
    logging.info("Mostrando las primeras 5 filas del DataFrame df_airbnb (formato markdown):")
    logging.info(f"\n{df_airbnb.head().to_markdown(index=False)}")
else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede mostrar el head.")

# Verificar Filas Duplicadas (df_airbnb)
logging.info("Verificando filas duplicadas en df_airbnb.")
if not df_airbnb.empty:
    num_duplicados = df_airbnb.duplicated().sum()
    logging.info(f"Número de filas duplicadas encontradas en df_airbnb: {num_duplicados}")

else:
    logging.warning("El DataFrame df_airbnb está vacío. No se pueden verificar duplicados.")
    print("El DataFrame df_airbnb está vacío.")

# Calcular Porcentaje de Nulos (df_airbnb)
logging.info("Calculando la cantidad y porcentaje de valores nulos por columna en df_airbnb.")
if not df_airbnb.empty:
    nulos_counts = df_airbnb.isnull().sum()
    nulos_percentage = (nulos_counts / len(df_airbnb)) * 100
    
    df_nulos = pd.DataFrame({
        'Columna': df_airbnb.columns,
        'Nulos': nulos_counts,
        'Porcentaje_Nulos': nulos_percentage
    })
    
    df_nulos_sorted = df_nulos[df_nulos['Nulos'] > 0].sort_values(by='Porcentaje_Nulos', ascending=False) # Mostrar solo columnas con nulos
    
    if not df_nulos_sorted.empty:
        print("Cantidad y porcentaje de valores nulos por columna (ordenado de mayor a menor, solo columnas con nulos):")
        print(df_nulos_sorted.to_markdown(index=False))
        logging.info("Tabla de nulos por columna generada y mostrada.")
    else:
        logging.info("No se encontraron valores nulos en df_airbnb.")
else:
    logging.warning("El DataFrame df_airbnb está vacío. No se pueden calcular los nulos.")

# Mostrar df_airbnb.info() con show_counts=False
logging.info("Mostrando información general de df_airbnb con df_airbnb.info(show_counts=False).")
if not df_airbnb.empty:
    print("\nInformación general de df_airbnb (show_counts=False):")
    df_airbnb.info(show_counts=False)
    logging.info("df_airbnb.info ejecutado.")

else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede mostrar .info().")

# Limpieza Preliminar y Conversión de Tipos de Datos (Volviendo a un estilo más conciso)
logging.info("Iniciando limpieza preliminar y conversión de tipos de datos (versión concisa).")
df_cleaned = pd.DataFrame() # Predefinir

if not df_airbnb.empty:
    # Crear una copia para no modificar el DataFrame original (df_airbnb)
    df_cleaned = df_airbnb.copy()
    logging.info("Copia de df_airbnb creada como df_cleaned.")

    # --- Paso Clave: Normalizar nombres de columnas en df_cleaned ---
    # Esto asegura que podemos usar nombres consistentes (ej. 'host_id') después.
    df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_', regex=False)
    logging.info(f"Columnas de df_cleaned normalizadas. Nuevas columnas: {df_cleaned.columns.tolist()}")

    try:
        # 1. id
        # Para el cálculo de nulos simplificado: contamos antes y después en df_cleaned
        nulls_before = df_cleaned['id'].isna().sum()
        df_cleaned['id'] = pd.to_numeric(df_cleaned['id'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['id'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'id': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'id' convertida a Int64.")

        # 2. name
        df_cleaned['name'] = df_cleaned['name'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        logging.info("Columna 'name' limpiada.")

        # 3. host_id
        nulls_before = df_cleaned['host_id'].isna().sum()
        df_cleaned['host_id'] = pd.to_numeric(df_cleaned['host_id'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['host_id'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'host_id': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'host_id' convertida a Int64.")

        # 4. host_identity_verified -> host_verification
        df_cleaned['host_verification'] = df_cleaned['host_identity_verified'].map({'verified': True, 'unconfirmed': False}).astype('boolean')
        logging.info("Columnas 'host_verification' creadas.")

        # 5. host_name
        df_cleaned['host_name'] = df_cleaned['host_name'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        logging.info("Columna 'host_name' limpiada.")

        # 6. neighbourhood_group
        df_cleaned['neighbourhood_group'] = df_cleaned['neighbourhood_group'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        # Opcional: convertir a category si hay pocos valores únicos
        if df_cleaned['neighbourhood_group'].nunique(dropna=False) < 20:
             df_cleaned['neighbourhood_group'] = df_cleaned['neighbourhood_group'].astype('category')
             logging.info("Columna 'neighbourhood_group' convertida a category.")
        else:
             logging.info("Columna 'neighbourhood_group' limpiada.")


        # 7. neighbourhood
        df_cleaned['neighbourhood'] = df_cleaned['neighbourhood'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        if df_cleaned['neighbourhood'].nunique(dropna=False) < 50:
             df_cleaned['neighbourhood'] = df_cleaned['neighbourhood'].astype('category')
             logging.info("Columna 'neighbourhood' convertida a category.")
        else:
            logging.info("Columna 'neighbourhood' limpiada.")

        # 8. lat
        nulls_before = df_cleaned['lat'].isna().sum()
        df_cleaned['lat'] = pd.to_numeric(df_cleaned['lat'], errors='coerce')
        coerced_nulls = df_cleaned['lat'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'lat': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'lat' convertida a float.")

        # 9. long
        nulls_before = df_cleaned['long'].isna().sum()
        df_cleaned['long'] = pd.to_numeric(df_cleaned['long'], errors='coerce')
        coerced_nulls = df_cleaned['long'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'long': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'long' convertida a float.")

        # 10. country
        df_cleaned['country'] = df_cleaned['country'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        if df_cleaned['country'].nunique(dropna=False) < 10:
            df_cleaned['country'] = df_cleaned['country'].astype('category')
            logging.info("Columna 'country' convertida a category.")
        else:
            logging.info("Columna 'country' limpiada.")


        # 11. country_code
        df_cleaned['country_code'] = df_cleaned['country_code'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        if df_cleaned['country_code'].nunique(dropna=False) < 10:
            df_cleaned['country_code'] = df_cleaned['country_code'].astype('category')
            logging.info("Columna 'country_code' convertida a category.")
        else:
            logging.info("Columna 'country_code' limpiada.")


        # 12. instant_bookable -> instant_bookable_flag
        df_cleaned['instant_bookable_flag'] = df_cleaned['instant_bookable'].map({'TRUE': True, 'FALSE': False, 'True': True, 'False': False, 'true': True, 'false': False}).astype('boolean')
        logging.info("Columna 'instant_bookable_flag' creada.")

        # 13. cancellation_policy
        df_cleaned['cancellation_policy'] = df_cleaned['cancellation_policy'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA}).astype('category')
        logging.info("Columna 'cancellation_policy' convertida a category.")

        # 14. room_type
        df_cleaned['room_type'] = df_cleaned['room_type'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA}).astype('category')
        logging.info("Columna 'room_type' convertida a category.")

        # 15. construction_year
        nulls_before = df_cleaned['construction_year'].isna().sum()
        df_cleaned['construction_year'] = pd.to_numeric(df_cleaned['construction_year'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['construction_year'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'construction_year': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'construction_year' convertida a Int64.")

        # 16. price -> price_str_cleaned, price_numeric
        df_cleaned['price_str_cleaned'] = df_cleaned['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().replace({'nan': pd.NA, '': pd.NA})
        df_cleaned['price_numeric'] = pd.to_numeric(df_cleaned['price_str_cleaned'], errors='coerce')
        logging.info("Columnas 'price_str_cleaned' y 'price_numeric' creadas.")

        # 17. service_fee -> service_fee_str_cleaned, service_fee_numeric
        df_cleaned['service_fee_str_cleaned'] = df_cleaned['service_fee'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().replace({'nan': pd.NA, '': pd.NA})
        df_cleaned['service_fee_numeric'] = pd.to_numeric(df_cleaned['service_fee_str_cleaned'], errors='coerce')
        logging.info("Columnas 'service_fee_str_cleaned' y 'service_fee_numeric' creadas.")

        # 18. minimum_nights
        nulls_before = df_cleaned['minimum_nights'].isna().sum()
        df_cleaned['minimum_nights'] = pd.to_numeric(df_cleaned['minimum_nights'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['minimum_nights'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'minimum_nights': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'minimum_nights' convertida a Int64.")

        # 19. number_of_reviews
        nulls_before = df_cleaned['number_of_reviews'].isna().sum()
        df_cleaned['number_of_reviews'] = pd.to_numeric(df_cleaned['number_of_reviews'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['number_of_reviews'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'number_of_reviews': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'number_of_reviews' convertida a Int64.")

        # 20. last_review
        nulls_before = df_cleaned['last_review'].isna().sum() # o .isnull()
        df_cleaned['last_review'] = pd.to_datetime(df_cleaned['last_review'], format='%m/%d/%Y', errors='coerce')
        coerced_nulls = df_cleaned['last_review'].isna().sum() - nulls_before # o .isnull()
        if coerced_nulls > 0:
            logging.warning(f"Columna 'last_review': {coerced_nulls} nuevos NaT por coerción.")
        logging.info("Columna 'last_review' convertida a datetime.")

        # 21. reviews_per_month
        nulls_before = df_cleaned['reviews_per_month'].isna().sum()
        df_cleaned['reviews_per_month'] = pd.to_numeric(df_cleaned['reviews_per_month'], errors='coerce')
        coerced_nulls = df_cleaned['reviews_per_month'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'reviews_per_month': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'reviews_per_month' convertida a float.")

        # 22. review_rate_number
        nulls_before = df_cleaned['review_rate_number'].isna().sum()
        df_cleaned['review_rate_number'] = pd.to_numeric(df_cleaned['review_rate_number'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['review_rate_number'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'review_rate_number': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'review_rate_number' convertida a Int64.")

        # 23. calculated_host_listings_count
        nulls_before = df_cleaned['calculated_host_listings_count'].isna().sum()
        df_cleaned['calculated_host_listings_count'] = pd.to_numeric(df_cleaned['calculated_host_listings_count'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['calculated_host_listings_count'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'calculated_host_listings_count': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'calculated_host_listings_count' convertida a Int64.")

        # 24. availability_365
        nulls_before = df_cleaned['availability_365'].isna().sum()
        df_cleaned['availability_365'] = pd.to_numeric(df_cleaned['availability_365'], errors='coerce').astype('Int64')
        coerced_nulls = df_cleaned['availability_365'].isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna 'availability_365': {coerced_nulls} nuevos NaNs por coerción.")
        logging.info("Columna 'availability_365' convertida a Int64.")

        # 25. house_rules
        df_cleaned['house_rules'] = df_cleaned['house_rules'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA})
        logging.info("Columna 'house_rules' limpiada.")

        # 26. license
        df_cleaned['license'] = df_cleaned['license'].astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA}) 
        logging.info("Columna 'license' limpiada.")

        # Columnas a eliminar (los nombres ya están normalizados en df_cleaned)
        cols_to_drop = ['host_identity_verified', 'instant_bookable', 'price', 'service_fee']
        existing_cols_to_drop = [col for col in cols_to_drop if col in df_cleaned.columns]
        if existing_cols_to_drop:
            df_cleaned.drop(columns=existing_cols_to_drop, inplace=True)
            logging.info(f"Columnas {existing_cols_to_drop} eliminadas de df_cleaned.")
        
        logging.info("Proceso de limpieza preliminar y conversión de tipos completado.")


    except KeyError as ke:
        logging.error(f"Ocurrió un KeyError: '{ke}'. Verifica que la columna exista en df_cleaned después de la normalización.")
        print(f"Ocurrió un KeyError: '{ke}'. Revisa los nombres de las columnas y la lógica de normalización.")
        print(f"Columnas disponibles en df_cleaned: {df_cleaned.columns.tolist()}")
    except Exception as e:
        logging.error(f"Ocurrió un error general durante la limpieza: {e}")
        import traceback
        logging.error(traceback.format_exc())
        print(f"Ocurrió un error general: {e}")
else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede realizar la limpieza.")
    print("El DataFrame df_airbnb está vacío.")


2025-05-17 18:15:48,670 - INFO - Inicio del notebook de EDA (002_eda_airbnb.ipynb).
2025-05-17 18:15:48,671 - INFO - Configuraciones de Pandas para visualización aplicadas.
2025-05-17 18:15:48,672 - INFO - Ruta del archivo CSV: ../../data/raw/Airbnb_Open_Data.csv
2025-05-17 18:15:48,674 - INFO - DataFrame 'df_airbnb' predefinido como un DataFrame vacío.
2025-05-17 18:15:48,675 - INFO - Intentando cargar el archivo CSV: ../../data/raw/Airbnb_Open_Data.csv
2025-05-17 18:15:49,338 - INFO - Archivo CSV '../../data/raw/Airbnb_Open_Data.csv' cargado exitosamente.
2025-05-17 18:15:49,340 - INFO - El DataFrame tiene 102599 filas y 26 columnas.
2025-05-17 18:15:49,341 - INFO - Mostrando las primeras 5 filas del DataFrame df_airbnb (formato markdown):
2025-05-17 18:15:49,350 - INFO - 
|      id | NAME                                             |     host id | host_identity_verified   | host name   | neighbourhood group   | neighbourhood   |     lat |     long | country       | country code   | 

Cantidad y porcentaje de valores nulos por columna (ordenado de mayor a menor, solo columnas con nulos):
| Columna                        |   Nulos |   Porcentaje_Nulos |
|:-------------------------------|--------:|-------------------:|
| license                        |  102597 |        99.9981     |
| house_rules                    |   52131 |        50.8104     |
| last review                    |   15893 |        15.4904     |
| reviews per month              |   15879 |        15.4768     |
| country                        |     532 |         0.518524   |
| availability 365               |     448 |         0.436651   |
| minimum nights                 |     409 |         0.398639   |
| host name                      |     406 |         0.395715   |
| review rate number             |     326 |         0.317742   |
| calculated host listings count |     319 |         0.310919   |
| host_identity_verified         |     289 |         0.281679   |
| service fee                    |   

2025-05-17 18:15:49,759 - INFO - Columna 'country' convertida a category.
2025-05-17 18:15:49,794 - INFO - Columna 'country_code' convertida a category.
2025-05-17 18:15:49,807 - INFO - Columna 'instant_bookable_flag' creada.
2025-05-17 18:15:49,836 - INFO - Columna 'cancellation_policy' convertida a category.
2025-05-17 18:15:49,866 - INFO - Columna 'room_type' convertida a category.
2025-05-17 18:15:49,876 - INFO - Columna 'construction_year' convertida a Int64.
2025-05-17 18:15:49,993 - INFO - Columnas 'price_str_cleaned' y 'price_numeric' creadas.
2025-05-17 18:15:50,112 - INFO - Columnas 'service_fee_str_cleaned' y 'service_fee_numeric' creadas.
2025-05-17 18:15:50,129 - INFO - Columna 'minimum_nights' convertida a Int64.
2025-05-17 18:15:50,141 - INFO - Columna 'number_of_reviews' convertida a Int64.
2025-05-17 18:15:50,167 - INFO - Columna 'last_review' convertida a datetime.
2025-05-17 18:15:50,169 - INFO - Columna 'reviews_per_month' convertida a float.
2025-05-17 18:15:50,178

In [ ]:
# Importación de Librerías
import pandas as pd
# import psycopg2 # Comentado ya que no se usa en este script específico
import os
import logging
import numpy as np
# import matplotlib.pyplot as plt # Comentado si no se usa EDA visual aquí
# import seaborn as sns # Comentado si no se usa EDA visual aquí
from rapidfuzz import process, fuzz # Para la limpieza con fuzzy matching

# Configuración del Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("002_eda_airbnb_optimized.log"), # Nuevo nombre de log
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook de EDA optimizado (002_eda_airbnb_optimized.py).")

# Configuraciones de Pandas para visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 1000)
logging.info("Configuraciones de Pandas para visualización aplicadas.")

# --- Extracción de Datos ---
# Definición de la ruta del archivo CSV usando os
# Asumiendo que el script está en un directorio y 'data/raw/' está dos niveles arriba
# Ajusta BASE_DIR según la estructura de tu proyecto
try:
    # Si se ejecuta como script .py
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Si se ejecuta en un entorno interactivo como Jupyter Notebook
    SCRIPT_DIR = os.getcwd()

BASE_DIR = os.path.dirname(os.path.dirname(SCRIPT_DIR)) # Sube dos niveles
RAW_DATA_DIR = os.path.join(BASE_DIR, 'data', 'raw')
CSV_FILE_NAME = 'Airbnb_Open_Data.csv'
CSV_FILE_PATH = os.path.join(RAW_DATA_DIR, CSV_FILE_NAME)

logging.info(f"Ruta del archivo CSV construida con os: {CSV_FILE_PATH}")

# Carga del DataFrame desde el archivo CSV
df_airbnb = pd.DataFrame()
logging.info("DataFrame 'df_airbnb' predefinido como un DataFrame vacío.")
try:
    logging.info(f"Intentando cargar el archivo CSV: {CSV_FILE_PATH}")
    if not os.path.exists(CSV_FILE_PATH):
        logging.error(f"Error: Archivo CSV no encontrado en '{CSV_FILE_PATH}'")
        raise FileNotFoundError(f"Archivo no encontrado: {CSV_FILE_PATH}")
    df_airbnb = pd.read_csv(CSV_FILE_PATH, low_memory=False)
    logging.info(f"Archivo CSV '{CSV_FILE_PATH}' cargado exitosamente.")
    logging.info(f"El DataFrame original tiene {df_airbnb.shape[0]} filas y {df_airbnb.shape[1]} columnas.")
except FileNotFoundError:
    # El error ya se logueó, simplemente relanzar para detener la ejecución si es crítico
    raise
except Exception as e:
    logging.error(f"Ocurrió un error al cargar el CSV '{CSV_FILE_PATH}': {e}")
    raise

# --- Exploración Inicial (sin cambios significativos, solo verificaciones) ---
if not df_airbnb.empty:
    logging.info("Verificando filas duplicadas en df_airbnb.")
    num_duplicados = df_airbnb.duplicated().sum()
    logging.info(f"Número de filas duplicadas encontradas en df_airbnb: {num_duplicados}")

    logging.info("Calculando la cantidad y porcentaje de valores nulos por columna en df_airbnb.")
    nulos_counts = df_airbnb.isnull().sum()
    nulos_percentage = (nulos_counts / len(df_airbnb)) * 100
    df_nulos = pd.DataFrame({
        'Columna': df_airbnb.columns,
        'Nulos': nulos_counts,
        'Porcentaje_Nulos': nulos_percentage
    })
    df_nulos_sorted = df_nulos[df_nulos['Nulos'] > 0].sort_values(by='Porcentaje_Nulos', ascending=False)
    if not df_nulos_sorted.empty:
        print("\nCantidad y porcentaje de valores nulos por columna (original, ordenado):")
        print(df_nulos_sorted.to_markdown(index=False))
        logging.info("Tabla de nulos por columna (original) generada y mostrada.")
    else:
        logging.info("No se encontraron valores nulos en el df_airbnb original.")

    logging.info("Mostrando información general de df_airbnb con df_airbnb.info(show_counts=False).")
    print("\nInformación general de df_airbnb (original, show_counts=False):")
    df_airbnb.info(show_counts=False)
    logging.info("df_airbnb.info (original) ejecutado.")
else:
    logging.critical("El DataFrame df_airbnb está vacío después de la carga. Terminando el script.")
    # Considerar salir del script si el DataFrame está vacío: exit()

# --- Limpieza Preliminar y Conversión de Tipos de Datos (Refactorizado) ---
logging.info("Iniciando limpieza preliminar y conversión de tipos de datos (versión optimizada).")
df_cleaned = pd.DataFrame()

if not df_airbnb.empty:
    df_cleaned = df_airbnb.copy()
    logging.info("Copia de df_airbnb creada como df_cleaned.")

    # --- Paso Clave: Normalizar nombres de columnas en df_cleaned ---
    original_columns = df_cleaned.columns.tolist()
    df_cleaned.columns = df_cleaned.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('[^0-9a-zA-Z_]', '', regex=True)
    new_columns = df_cleaned.columns.tolist()
    logging.info(f"Columnas de df_cleaned normalizadas.")
    if original_columns != new_columns:
        logging.info(f"Cambios en nombres de columnas: {dict(zip(original_columns, new_columns))}")
    else:
        logging.info("Nombres de columnas ya estaban normalizados o no requirieron cambios significativos.")


    # Diccionario para mapear nombres originales (si cambiaron) a los nuevos normalizados
    # Esto es crucial si los nombres originales tenían caracteres especiales eliminados
    # Ejemplo: 'neighbourhood group' -> 'neighbourhood_group'
    #          'price$' -> 'price'
    # La normalización de arriba ya los debería tener como 'price' si había un '$'
    # Re-evaluar si esta normalización es la deseada para todos los casos o si se necesita
    # un mapeo más explícito si la normalización es muy agresiva.
    # Por ahora, la normalización str.lower().str.replace(' ', '_') es la principal.

    # Funciones auxiliares para limpieza
    def clean_string_column(series, col_name):
        logging.debug(f"Limpiando columna string: {col_name}")
        series = series.astype(str).str.strip().replace({'nan': pd.NA, '': pd.NA, 'None': pd.NA})
        return series

    def to_numeric_column(series, col_name, numeric_type='Int64'):
        logging.debug(f"Convirtiendo columna a numérica ({numeric_type}): {col_name}")
        nulls_before = series.isna().sum()
        if numeric_type == 'datetime':
            # Asumiendo un formato común, ajustar si es necesario
            series = pd.to_datetime(series, format='%m/%d/%Y', errors='coerce')
        else:
            series = pd.to_numeric(series, errors='coerce')
            if numeric_type == 'Int64' and not series.empty: # Asegurar que Int64 solo se aplica si no está vacío
                 # Verificar si después de pd.to_numeric quedan solo nulos y números enteros
                if series.dropna().apply(lambda x: x.is_integer()).all() or series.dropna().empty:
                    series = series.astype('Int64') # Usa el tipo Int64 de Pandas que soporta NA
                else:
                    logging.warning(f"Columna '{col_name}' contiene flotantes, no se convertirá a Int64, se mantendrá como float.")
            # Para float, pd.to_numeric ya devuelve float si hay decimales.
        
        coerced_nulls = series.isna().sum() - nulls_before
        if coerced_nulls > 0:
            logging.warning(f"Columna '{col_name}': {coerced_nulls} nuevos NaNs/NaTs por coerción.")
        return series

    # Función para estandarizar categóricas usando RapidFuzz (EJEMPLO)
    def standardize_categorical_fuzz(series, col_name, choices_list, score_cutoff=85):
        logging.debug(f"Estandarizando columna categórica con RapidFuzz: {col_name}")
        # Asegurarse de que la serie es de tipo string para el mapeo
        # y que los NaN se manejan correctamente (rapidfuzz puede no gustarle pd.NA directamente en map)
        
        # Crear un mapeo
        unique_values = series.dropna().unique()
        mapping = {}
        for val in unique_values:
            match = process.extractOne(str(val), choices_list, scorer=fuzz.WRatio, score_cutoff=score_cutoff)
            if match:
                mapping[val] = match[0] # Mapear al valor canónico
            else:
                mapping[val] = val # Mantener original si no hay buen match
        
        # Aplicar el mapeo, asegurándose de manejar NaNs
        original_na_mask = series.isna()
        series_mapped = series.map(mapping)
        series_mapped[original_na_mask] = pd.NA # Re-aplicar NaNs
        
        # Contar cuántos valores cambiaron
        changes = (series.dropna() != series_mapped.dropna()).sum()
        if changes > 0:
            logging.info(f"Columna '{col_name}': {changes} valores estandarizados usando RapidFuzz.")
        return series_mapped

    try:
        # Aplicar transformaciones
        # Numéricas (Int64 o float según corresponda)
        numeric_cols_int = ['id', 'host_id', 'construction_year', 'minimum_nights', 
                            'number_of_reviews', 'review_rate_number', 
                            'calculated_host_listings_count', 'availability_365']
        for col in numeric_cols_int:
            if col in df_cleaned.columns:
                df_cleaned[col] = to_numeric_column(df_cleaned[col], col, 'Int64')
            else: logging.warning(f"Columna '{col}' no encontrada para conversión numérica (Int64).")

        numeric_cols_float = ['lat', 'long', 'reviews_per_month']
        for col in numeric_cols_float:
            if col in df_cleaned.columns:
                df_cleaned[col] = to_numeric_column(df_cleaned[col], col, 'float')
            else: logging.warning(f"Columna '{col}' no encontrada para conversión numérica (float).")

        # Strings
        string_cols = ['name', 'host_name', 'house_rules', 'license']
        for col in string_cols:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col)
            else: logging.warning(f"Columna '{col}' no encontrada para limpieza de string.")

        # Categóricas y Fuzzy Matching (Ejemplo con 'neighbourhood_group')
        # Primero, limpieza básica de string
        categorical_cols_pre_fuzz = ['neighbourhood_group', 'neighbourhood', 'country', 'country_code']
        for col in categorical_cols_pre_fuzz:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col)
                
                # EJEMPLO DE USO DE RAPIDFUZZ (DESCOMENTAR Y ADAPTAR SI ES NECESARIO)
                # Esto es más útil si tienes una lista de valores canónicos conocidos
                # o si quieres consolidar valores muy similares.
                # if col == 'neighbourhood_group' and col in df_cleaned.columns:
                #     # Podrías obtener `canonical_groups` de df_cleaned[col].value_counts().nlargest(5).index.tolist()
                #     # o de una lista externa. Aquí un ejemplo hardcodeado:
                #     canonical_groups = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']
                #     if not df_cleaned[col].dropna().empty: # Solo si hay datos no nulos
                #        df_cleaned[col] = standardize_categorical_fuzz(df_cleaned[col], col, canonical_groups, score_cutoff=80)
                #        logging.info(f"RapidFuzz aplicado a '{col}'.")
                #     else:
                #        logging.info(f"Columna '{col}' está vacía o solo nulos, RapidFuzz no aplicado.")

                # Convertir a category después de la limpieza (y opcionalmente fuzzy)
                # El umbral de nunique es una heurística
                nunique_threshold = 50 if col == 'neighbourhood' else 20 
                if col in df_cleaned.columns and df_cleaned[col].nunique(dropna=False) < nunique_threshold:
                    df_cleaned[col] = df_cleaned[col].astype('category')
                    logging.info(f"Columna '{col}' convertida a category.")
                elif col in df_cleaned.columns:
                     logging.info(f"Columna '{col}' limpiada (no convertida a category debido a alta cardinalidad: {df_cleaned[col].nunique(dropna=False)}).")

            else: logging.warning(f"Columna '{col}' no encontrada para limpieza categórica.")

        # Categóricas directas
        category_cols_direct = ['cancellation_policy', 'room_type']
        for col in category_cols_direct:
            if col in df_cleaned.columns:
                df_cleaned[col] = clean_string_column(df_cleaned[col], col).astype('category')
                logging.info(f"Columna '{col}' convertida a category.")
            else: logging.warning(f"Columna '{col}' no encontrada para conversión a category.")
        
        # Fechas
        if 'last_review' in df_cleaned.columns:
            df_cleaned['last_review'] = to_numeric_column(df_cleaned['last_review'], 'last_review', 'datetime')
            logging.info("Columna 'last_review' convertida a datetime.")
        else: logging.warning("Columna 'last_review' no encontrada.")

        # Booleanas (con mapeo)
        if 'host_identity_verified' in df_cleaned.columns:
            verified_map = {'verified': True, 'unconfirmed': False}
            df_cleaned['host_verification'] = df_cleaned['host_identity_verified'].map(verified_map).astype('boolean')
            df_cleaned['is_unconfirmed'] = df_cleaned['host_identity_verified'].map({k: not v for k, v in verified_map.items() if pd.notna(v)}).astype('boolean') # Invierte el mapeo para unconfirmed
            logging.info("Columnas 'host_verification', 'is_unconfirmed' creadas a partir de 'host_identity_verified'.")
        else: logging.warning("Columna 'host_identity_verified' no encontrada.")

        if 'instant_bookable' in df_cleaned.columns:
            bookable_map = {'TRUE': True, 'FALSE': False, 'True': True, 'False': False, 'true': True, 'false': False} # Cubrir varias capitalizaciones
            df_cleaned['instant_bookable_flag'] = df_cleaned['instant_bookable'].astype(str).str.upper().map(bookable_map).astype('boolean')
            logging.info("Columna 'instant_bookable_flag' creada a partir de 'instant_bookable'.")
        else: logging.warning("Columna 'instant_bookable' no encontrada.")

        # Columnas de precio y tarifa de servicio
        currency_cols = {'price': 'price_numeric', 'service_fee': 'service_fee_numeric'}
        for original_col, new_col_numeric in currency_cols.items():
            if original_col in df_cleaned.columns:
                # Crear columna intermedia _str_cleaned solo si es necesario para el log o inspección
                # df_cleaned[f'{original_col}_str_cleaned'] = df_cleaned[original_col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().replace({'nan': pd.NA, '': pd.NA})
                # df_cleaned[new_col_numeric] = to_numeric_column(df_cleaned[f'{original_col}_str_cleaned'], new_col_numeric, 'float')
                
                # Versión más directa:
                series_cleaned_str = df_cleaned[original_col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().replace({'nan': pd.NA, '': pd.NA})
                df_cleaned[new_col_numeric] = to_numeric_column(series_cleaned_str, new_col_numeric, 'float')
                logging.info(f"Columna '{new_col_numeric}' creada a partir de '{original_col}'.")
            else: logging.warning(f"Columna original '{original_col}' no encontrada para procesar moneda.")

        # Columnas a eliminar (después de que sus datos se hayan usado para crear nuevas columnas)
        # Asegurarse que los nombres de las columnas aquí son los normalizados si se modificaron
        # o los originales si la normalización no los afectó
        cols_to_drop_original_names = ['host_identity_verified', 'instant_bookable', 'price', 'service_fee', 'country', 'country_code', 'license', 'house_rules']
        
        # Mapear nombres originales a normalizados para la eliminación
        # Esto asume una normalización simple. Si fue más compleja, ajustar.
        normalized_cols_to_drop = []
        for col_name in cols_to_drop_original_names:
            normalized_name = col_name.lower().replace(' ', '_').replace('[^0-9a-zA-Z_]', '')
            if normalized_name in df_cleaned.columns:
                 normalized_cols_to_drop.append(normalized_name)
            elif col_name in df_cleaned.columns: # Si la normalización no cambió el nombre
                 normalized_cols_to_drop.append(col_name)


        # Eliminar solo las columnas que realmente existen en df_cleaned
        existing_cols_to_drop = [col for col in normalized_cols_to_drop if col in df_cleaned.columns]
        if existing_cols_to_drop:
            df_cleaned.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
            logging.info(f"Columnas {existing_cols_to_drop} eliminadas de df_cleaned.")
        
        logging.info("Proceso de limpieza preliminar y conversión de tipos optimizado completado.")

    except KeyError as ke:
        logging.error(f"Ocurrió un KeyError durante la limpieza: '{ke}'. Verifica que la columna exista en df_cleaned (posiblemente después de la normalización).")
        logging.error(f"Columnas disponibles en df_cleaned: {df_cleaned.columns.tolist()}")
        print(f"Ocurrió un KeyError: '{ke}'. Revisa los nombres de las columnas y la lógica de normalización.")
    except Exception as e:
        logging.error(f"Ocurrió un error general durante la limpieza: {e}", exc_info=True)
        print(f"Ocurrió un error general: {e}")

else:
    logging.warning("El DataFrame df_airbnb está vacío. No se puede realizar la limpieza.")
    print("El DataFrame df_airbnb está vacío.")


# --- Verificación Post-Limpieza ---
if not df_cleaned.empty:
    logging.info("Calculando la cantidad y porcentaje de valores nulos por columna en df_cleaned.")
    nulos_counts_cleaned = df_cleaned.isnull().sum()
    nulos_percentage_cleaned = (nulos_counts_cleaned / len(df_cleaned)) * 100
    
    df_nulos_cleaned = pd.DataFrame({
        'Columna': df_cleaned.columns,
        'Nulos': nulos_counts_cleaned,
        'Porcentaje_Nulos': nulos_percentage_cleaned
    })
    
    df_nulos_cleaned_sorted = df_nulos_cleaned[df_nulos_cleaned['Nulos'] > 0].sort_values(by='Porcentaje_Nulos', ascending=False)
    
    if not df_nulos_cleaned_sorted.empty:
        print("\nCantidad y porcentaje de valores nulos por columna en df_cleaned (ordenado, solo con nulos):")
        print(df_nulos_cleaned_sorted.to_markdown(index=False))
        logging.info("Tabla de nulos por columna (df_cleaned) generada y mostrada.")
    else:
        logging.info("No se encontraron valores nulos en df_cleaned.")

    logging.info("Mostrando información general de df_cleaned con df_cleaned.info(show_counts=True).") # show_counts=True para ver nulos
    print("\nInformación general de df_cleaned (show_counts=True):")
    df_cleaned.info(show_counts=True)
    logging.info("df_cleaned.info ejecutado.")
else:
    logging.warning("El DataFrame df_cleaned está vacío. No se pueden mostrar resultados post-limpieza.")

logging.info("Fin del notebook de EDA optimizado.")

2025-05-17 18:23:38,308 - INFO - Inicio del notebook de EDA optimizado (002_eda_airbnb_optimized.py).
2025-05-17 18:23:38,310 - INFO - Configuraciones de Pandas para visualización aplicadas.
2025-05-17 18:23:38,311 - INFO - Ruta del archivo CSV construida con os: /home/jacobo/Proyecto_ETL/data/raw/Airbnb_Open_Data.csv
2025-05-17 18:23:38,316 - INFO - DataFrame 'df_airbnb' predefinido como un DataFrame vacío.
2025-05-17 18:23:38,318 - INFO - Intentando cargar el archivo CSV: /home/jacobo/Proyecto_ETL/data/raw/Airbnb_Open_Data.csv
2025-05-17 18:23:37,049 - INFO - Archivo CSV '/home/jacobo/Proyecto_ETL/data/raw/Airbnb_Open_Data.csv' cargado exitosamente.
2025-05-17 18:23:37,050 - INFO - El DataFrame original tiene 102599 filas y 26 columnas.
2025-05-17 18:23:37,050 - INFO - Verificando filas duplicadas en df_airbnb.
2025-05-17 18:23:37,187 - INFO - Número de filas duplicadas encontradas en df_airbnb: 541
2025-05-17 18:23:37,188 - INFO - Calculando la cantidad y porcentaje de valores nulos


Cantidad y porcentaje de valores nulos por columna (original, ordenado):
| Columna                        |   Nulos |   Porcentaje_Nulos |
|:-------------------------------|--------:|-------------------:|
| license                        |  102597 |        99.9981     |
| house_rules                    |   52131 |        50.8104     |
| last review                    |   15893 |        15.4904     |
| reviews per month              |   15879 |        15.4768     |
| country                        |     532 |         0.518524   |
| availability 365               |     448 |         0.436651   |
| minimum nights                 |     409 |         0.398639   |
| host name                      |     406 |         0.395715   |
| review rate number             |     326 |         0.317742   |
| calculated host listings count |     319 |         0.310919   |
| host_identity_verified         |     289 |         0.281679   |
| service fee                    |     273 |         0.266084   |
| 

2025-05-17 18:23:37,680 - INFO - Columna 'neighbourhood_group' convertida a category.
2025-05-17 18:23:37,713 - INFO - Columna 'neighbourhood' limpiada (no convertida a category debido a alta cardinalidad: 225).
2025-05-17 18:23:37,749 - INFO - Columna 'country' convertida a category.
2025-05-17 18:23:37,784 - INFO - Columna 'country_code' convertida a category.
2025-05-17 18:23:37,819 - INFO - Columna 'cancellation_policy' convertida a category.
2025-05-17 18:23:37,859 - INFO - Columna 'room_type' convertida a category.
2025-05-17 18:23:37,884 - INFO - Columna 'last_review' convertida a datetime.
2025-05-17 18:23:37,920 - INFO - Columnas 'is_verified', 'is_unconfirmed' creadas a partir de 'host_identity_verified'.
2025-05-17 18:23:37,990 - INFO - Columna 'instant_bookable_flag' creada a partir de 'instant_bookable'.
2025-05-17 18:23:38,133 - INFO - Columna 'price_numeric' creada a partir de 'price'.
2025-05-17 18:23:38,269 - INFO - Columna 'service_fee_numeric' creada a partir de 'ser


Cantidad y porcentaje de valores nulos por columna en df_cleaned (ordenado, solo con nulos):
| Columna                        |   Nulos |   Porcentaje_Nulos |
|:-------------------------------|--------:|-------------------:|
| license                        |  102597 |        99.9981     |
| house_rules                    |   52131 |        50.8104     |
| last_review                    |   15893 |        15.4904     |
| reviews_per_month              |   15879 |        15.4768     |
| country                        |     532 |         0.518524   |
| availability_365               |     448 |         0.436651   |
| minimum_nights                 |     409 |         0.398639   |
| host_name                      |     406 |         0.395715   |
| review_rate_number             |     326 |         0.317742   |
| calculated_host_listings_count |     319 |         0.310919   |
| is_unconfirmed                 |     289 |         0.281679   |
| is_verified                    |     289 |    

In [9]:
df_airbnb.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,country code,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.647,-73.972,United States,US,False,strict,Private room,2020.000,$966,$193,10.000,9.000,10/19/2021,0.210,4.000,6.000,286.000,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.754,-73.984,United States,US,False,moderate,Entire home/apt,2007.000,$142,$28,30.000,45.000,5/21/2022,0.380,4.000,2.000,228.000,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.809,-73.942,United States,US,True,flexible,Private room,2005.000,$620,$124,3.000,0.000,NaN,NaN,5.000,1.000,352.000,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.685,-73.960,United States,US,True,moderate,Entire home/apt,2005.000,$368,$74,30.000,270.000,7/5/2019,4.640,4.000,1.000,322.000,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.799,-73.944,United States,US,False,moderate,Entire home/apt,2009.000,$204,$41,10.000,9.000,11/19/2018,0.100,3.000,1.000,289.000,"Please no smoking in the house, porch or on th...",NaN


In [10]:
df_cleaned.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,lat,long,country,country_code,cancellation_policy,room_type,construction_year,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,is_verified,is_unconfirmed,instant_bookable_flag,price_numeric,service_fee_numeric
0,1001254,Clean & quiet apt home by the park,80014485718,Madaline,Brooklyn,Kensington,40.647,-73.972,United States,US,strict,Private room,2020,10,9,2021-10-19,0.210,4,6,286,Clean up and treat the home the way you'd like...,<NA>,False,True,False,966.000,193.000
1,1002102,Skylit Midtown Castle,52335172823,Jenna,Manhattan,Midtown,40.754,-73.984,United States,US,moderate,Entire home/apt,2007,30,45,2022-05-21,0.380,4,2,228,Pet friendly but please confirm with me if the...,<NA>,True,False,False,142.000,28.000
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,Elise,Manhattan,Harlem,40.809,-73.942,United States,US,flexible,Private room,2005,3,0,NaT,NaN,5,1,352,"I encourage you to use my kitchen, cooking and...",<NA>,<NA>,<NA>,True,620.000,124.000
3,1002755,<NA>,85098326012,Garry,Brooklyn,Clinton Hill,40.685,-73.960,United States,US,moderate,Entire home/apt,2005,30,270,2019-07-05,4.640,4,1,322,<NA>,<NA>,False,True,True,368.000,74.000
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,Lyndon,Manhattan,East Harlem,40.799,-73.944,United States,US,moderate,Entire home/apt,2009,10,9,2018-11-19,0.100,3,1,289,"Please no smoking in the house, porch or on th...",<NA>,True,False,False,204.000,41.000


In [19]:
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['id'] = df_cleaned.index + 1

df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['host_id'] = df_cleaned.index + 150000

In [15]:
len(df_airbnb)

102599

In [20]:
df_cleaned.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,lat,long,country,country_code,cancellation_policy,room_type,construction_year,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,is_verified,is_unconfirmed,instant_bookable_flag,price_numeric,service_fee_numeric,fsq_id,host
0,1,Clean & quiet apt home by the park,150000,Madaline,Brooklyn,Kensington,40.647,-73.972,United States,US,strict,Private room,2020,10,9,2021-10-19,0.210,4,6,286,Clean up and treat the home the way you'd like...,<NA>,False,True,False,966.000,193.000,1,150000
1,2,Skylit Midtown Castle,150001,Jenna,Manhattan,Midtown,40.754,-73.984,United States,US,moderate,Entire home/apt,2007,30,45,2022-05-21,0.380,4,2,228,Pet friendly but please confirm with me if the...,<NA>,True,False,False,142.000,28.000,2,150001
2,3,THE VILLAGE OF HARLEM....NEW YORK !,150002,Elise,Manhattan,Harlem,40.809,-73.942,United States,US,flexible,Private room,2005,3,0,NaT,NaN,5,1,352,"I encourage you to use my kitchen, cooking and...",<NA>,<NA>,<NA>,True,620.000,124.000,3,150002
3,4,<NA>,150003,Garry,Brooklyn,Clinton Hill,40.685,-73.960,United States,US,moderate,Entire home/apt,2005,30,270,2019-07-05,4.640,4,1,322,<NA>,<NA>,False,True,True,368.000,74.000,4,150003
4,5,Entire Apt: Spacious Studio/Loft by central park,150004,Lyndon,Manhattan,East Harlem,40.799,-73.944,United States,US,moderate,Entire home/apt,2009,10,9,2018-11-19,0.100,3,1,289,"Please no smoking in the house, porch or on th...",<NA>,True,False,False,204.000,41.000,5,150004
